<img src="assets/bootcamp.png">

# **LTI&trade; Advantage** bootcamp notebook for Tool

Ping me if you have any question or feedback: claude.vervoort@gmail.com

This bootcamp code is available on: https://github.com/claudevervoort-perso/ltibootcamp, if you want to run it on your local laptop/network.


## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a learning tool implementer viewpoint. It interacts with an actual test server acting as an LTI Platform simumlator to support this notebook.

If you want a refreshed on what is LTI and LTI Advantage, you can check the 9 minutes video here or on [youtube](https://www.youtube.com/watch?v=f_6pWiQpg5s).


In [6]:
from IPython.display import display, Markdown, HTML, Javascript, IFrame

display(IFrame(src='https://www.youtube.com/embed/f_6pWiQpg5s', width=560, height=315))

## Limitation

<img src="assets/bootcamp_arch.png" width="60%">

The test tool platform cannot launch into the bootcamp. So we will initiate the LTI launches directly rather than having those triggered by a call from the platform on a 3rd party initiated login end point.

# Setup

## Libraries and utilities

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [1]:
import requests
import json
import jwt
import base64
import re
from random import randint
from time import time, sleep
from datetime import datetime, timedelta
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim 
def fc(claim, s=None):
    suffix = ('-'+s) if s else ''
    return 'https://purl.imsglobal.org/spec/lti{0}/claim/{1}'.format(suffix, claim)

def scope(scope, s=None):
    suffix = ('-'+s) if s else ''
    return 'https://purl.imsglobal.org/spec/lti{0}/scope/{1}'.format(suffix, scope)
    
def md(mdt):
    display(Markdown(mdt))
    
def mdjson(j):
    md("```{0}```".format(j))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''
        
# In LTI (and OIDC flows) End user's browser gets redirected from 
# the platform to the tool (or vice versa) using Autopost forms
# i.e pages that add a form and submits it right away.
# Since we cannot redirect the full notebook, those interactions
# happen in IFrames.
def autopost_iframe(name, url, params):
    inputs = ''
    for key in params.keys():
        input = """$('<input>').attr({{
                    type: 'hidden',
                    name: '{name}',
                    value: '{value}'
                }}).appendTo(ltiForm);
                """.format(name=key, value=params[key])
        inputs = inputs + input
        autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', '{name}');
                {inputs}
                $('#{name}').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

    autosubmit_js = autosubmit_js.format(name=name, url=url, inputs=inputs)

    #md(autosubmit_js)
    display(HTML('<iframe id="{name}" name="{name}" style="height: 300px; width:100%"></iframe>'.format(name=name)))
    display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))



## LTI Platform url

In [10]:
# This notebook queries an actual tool platform test server. It needs its location
# if you run the server locally you can use
# platform_url='http://localhost:5000'
platform_url='https://ltiplatform.theedtech.dev'

r = requests.get(platform_url + '/ping')
if r.text=='pong!':
    md('Platform url set and verified')
else:
    md('Check platform url')

SSLError: HTTPSConnectionPool(host='ltiplatform.theedtech.dev', port=443): Max retries exceeded with url: /ping (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))

# Call back to this notebook

The platform needs to send LTI launches data to this notebook. It is started with an extension to receive that data and display it. This is checking the URL is correct and the extension set.

In [7]:
 %%javascript
var kernel = IPython.notebook.kernel
var host = window.location.href.split('/notebooks')[0]
kernel.execute("notebook_url='"+host+"'")


<IPython.core.display.Javascript object>

In [11]:
r = requests.get(notebook_url + '/ping')
if r.text=='pong!':
    md('Notebook url set and verified')
else:
    md('Check notebook url, possibly set it manually. Check Jupyter was started with extension.')

Notebook url set and verified

# Registering a new tool

Trust in LTi 1.3 is based on Public/Private keys. See [Trust in LTI Advantage](https://youtu.be/bWMVneE7vqI).

In [12]:
display(IFrame(width=560, height=315, src='https://www.youtube.com/embed/bWMVneE7vqI'))



## Deploying tool and establishing keys

### get the tool deployment info to use in this notebook

First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a `client_id` and a private key that will be used to interact with the platform services and send messages back to the platform. The `client_id` is used to for security purposes. A trust may be shared across multiple deployments of the same tool in a platform, so a `deployment_id` is also communicated to identify the actual deployment of that tool.

It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

<img src="assets/security.png" width="70%">

#### Compute a Private/Public key pair for this notebook

In [8]:
notebook_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

notebook_public_key = notebook_private_key.public_key()

public_key_pem = notebook_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo)


mdb("Private key asserts the **identity** of the tool. It should be stored securely and ")
mdb("ideally it should not be possible for a human to even see it, even less share it!")
mdb("However the public key can be shared around, here is the public in the common")
mdb("PEM format that identifies this notebook instance:")
mdb()
mdjson(public_key_pem.decode('utf-8'))


Private key asserts the **identity** of the tool. It should be stored securely and 
ideally it should not be possible for a human to even see it, even less share it!
However the public key can be shared around, here is the public in the common
PEM format that identifies this notebook instance:

```-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAsSR71w2wgxUc3rc4OVGm
32/H1IXpJ36cMQYZm1//6UBs+xq//3a1SUumFiJUEt2DvGOuQlnCaK/NhwWdPpRv
Ix3c8sBJde4I1h23nfh9Whhx86XuNKhkJSudsat2axnA7GmMKgDsL+HoHALtxn7u
hoTaZrbqnZc/5dEqfFztJu7+4TjX1qnCzVEZ+ePqm2LQeWGptShsqsKNgFdhfEg1
wwvSpprta8kB4+vpKpm5b+PQ39IwEXHbS1flMlokNlw3HbvTashpTb6Chg9MTGo2
JQDPE/ACfXR28fUNxvO6ZONpvhBcvPGsd00yH4WflSDZvinRQM/ICHbM0PU+vHOL
hQIDAQAB
-----END PUBLIC KEY-----
```

#### Redirect URI

We need to register the Redirect URI to where the OpenId Connect response can be returned. While this notebook only register a single `redirect_uri` it is common for tools to register more than one as platforms are discouraged to allow pattern matching for those (i.e. a platform should always check for an exact match)

This URL is specific to this notebook instance which is extended with a handler to display the actual authentication response from where you can copy the actual token value.

In [18]:
redirect_uri = "{0}/auth".format(notebook_url)

md("redirect URI is "+redirect_uri)

redirect URI is http://localhost:8888/auth

In [19]:
registration_data = {
    'public_key_pem':public_key_pem,
    'redirect_uris': redirect_uri #comma separated list of redirect_uris
}

r = requests.post(platform_url + '/newtool', data=registration_data, verify=False)
tool_info = json.loads(r.text)

md('### Tool registration information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
mdjson(json.dumps(tool_info, indent=4))

/home/claude/py3envs/jupyter/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


### Tool registration information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```{
    "accesstoken_endpoint": "http://ltiplatform.theedtech.dev/auth/token",
    "client_id": "1",
    "keyset_url": "http://ltiplatform.theedtech.dev/.well-known/jwks.json",
    "oidc_auth_endpoint": "http://ltiplatform.theedtech.dev/auth"
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [20]:
keyset = json.loads(requests.get(tool_info['keyset_url']).text)

md('#### Platform keyset')
mdjson(json.dumps(keyset, indent=4))

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```{
    "keys": [
        {
            "alg": "RS256",
            "e": "AQAB",
            "kid": "1557603581_0",
            "kty": "RSA",
            "n": "xnWllQaA71H3cJE1Neyetj3AxB--KStP0rcE1CwHF6ZBlX8EaqdpwdaScroTpoK6nd17hw-lHPTu3D76E0FkhuJGxrofDap2JCRwFrn9Fuuouhjk7DtIXQgfdZnS2UaJb30hg-9brUHLLmcOsEjdzvFDn9kDlTdKTT_yZx_Rb9HgfbM8NH3kMkbEAkzhUKiYeSKCv45cWRErx1zl91gtZCvOcHU65JXWSm3ChkMEHg2pZ2LgsEFP0hz5JCaOgGEN67kNgzIcAk4iITJ7dql_Olx6voJcbQgcpBt_A4oVaFwvS8B2CMP2zV0TixnVRHmSyVuVxVrnx-v0WwZqxUIO1w==",
            "use": "sig"
        },
        {
            "alg": "RS256",
            "e": "AQAB",
            "kid": "1557603581_1",
            "kty": "RSA",
            "n": "lzbXcJJhhxjuo1JPBDTZPIS0gh2jFA2pA3eY_BLXeaAWLdpFZL1F7THiflYrIlI92phfcffWY70LjDh1Pk_R-gIL2xUrQg_yW4H9zs5DANWhrRevwcDKtSUfivFu52_W8W7Nmdj_ObncQWdCXll7JasRUqHYB9UL1Tst_CoVsh-KhOsmmVbmYtL6FP2yfjUtsxD4Pbpxlf6zzYIuAXAaUbH8kC8V6Pn60WjkVcTYZ7AFXkMmg-ub_lhZvucLEK87mVoE6tcDWV6uukAdvM84PaK2tIiMPriqNumJflxDyE-J132UjcUvHuV7aQraXsTpQuF_sHDzPD4j3PTU5Pm2HQ==",
            "use": "sig"
        },
        {
            "alg": "RS256",
            "e": "AQAB",
            "kid": "1557603581_2",
            "kty": "RSA",
            "n": "p3VGnGHGhHyMnJpF0T6_AQ4UrFKEq86MHewmdIkMQ25MZFe9rkyKP36UzyodJR5KcRk4qlbOWk4OWIaYldk4HD3_fzvAqoeP01tiu9_V-WjbZlLX8joz7Vp6nScQmHriWtMyk4tL8lvTHiINHzBl3Fi28YsejJzAanJFMe8xSeQaCf9NxlmJBHdOU8Ih5W0XB1jtPlvQippKuY6wZ0cCuvfSbkEIHzHZQOBd_zh2JKkza5bMiEMGR48Gp_by4qTAGQqhWUCFa6zIeCrXLNgNtjIdS8HQZ5bMNcK-Gm14dSn3QYhqrnHDnJNOACAlMq8tBJpVKe_5ppPia1JH_eY-bw==",
            "use": "sig"
        },
        {
            "alg": "RS256",
            "e": "AQAB",
            "kid": "1557603581_3",
            "kty": "RSA",
            "n": "1s_qs-WciZWTtW1Zh2i8l7RW9-SO_uhW-wYKImwCnaVti97KmbdKEctv2qML7q6qBAKhabmghESbCZUYCacLKmFE1SFtYhsH3InCuaYB3V3lOgSlGfgD8vAzjbXpeYE3gmMqSn3Z-_pXUAuY7Z6QpGJWxzXOnnDZoDpHYK1O6wPiFr9x2F2V4cRyWcfmNOOOOLYQWoguipg-rtBUwHKTgFb3vVNSVY54hz5_Jk_WfUpXHznVoFnL9ozxSMYnpGhIchdh-e_shf2N2JBkFE09C7LeUnj6vsc4vWwqqf4J_SGMBdNEYD4q3s0wDp-O4DsJ03h_G9dP_O3n6_0snTCZ-w==",
            "use": "sig"
        }
    ]
}```

# LTI Launches and OpenId Connect 

LTI Launches are now done in 2 steps: 

1. a pre-flight indicating to the tool a launch is intended to happen. The tool indicates where that launch data should be sent (`redirect_uri`) and also mark the browser that will make this request to avoid cross-site request forgery, which the LTI 1.1 family is susceptible to. 
2. Then the learning platform issues the lti launch payload in the form on an `id_token` back to the tool. The tool can correlate that request with the browser marker to make sure the launch is happening from a browser that requested it.
If all that sounds vaguely familiar, it's no accident: the LTI Launch is now following the 3rd Party initiated login from OpenId Connect.

This is summarized in this diagram. Note that this notebook cannt be launched from the platform, so we simulate the 1st leg in the launches below.

<img src="assets/ltioidcflow.png" width="70%">

You can also check the [LTI Launch and OIDC video](https://youtu.be/g3y4vwtP6vQ).

In [13]:
display(IFrame(width=560, height=315, src='https://www.youtube.com/embed/g3y4vwtP6vQ'))


# Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `LtiDeepLinkingRequest` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `LtiDeepLinkingResponse` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

<img src="assets/deeplinking.png" width="60%">

### Setup: Getting a Deep Linking Request

This notebook is not actually launched by the platform, so it is not getting the initiate login request that usually  bootstraps the LTI message flow. Instead we will just pretend we got that request and the 2 parameters it contains, and just build those locally.

We also compute a `state` and a `nonce` we will pass along that will serve as the Cross-Site Scripting protection (XSRF). They  must be tied to the end user's browser session. The use of `state` is optional, `nonce` is required. `state` will be passed back as separate parameter while `nonce` will be included in the token. `nonce` should not be re-used in subsequent requests as they uniquely identfy an authentication request.

In [21]:
# Prentending this is what we received from the Platform to initiate the deep linking flow
init_login_request = {
    'iss': platform_url,
    'login_hint': 'instructor', #value is opaque to tool, ex: tie to end user's platform's session
    'lti_message_hint': 'deeplink', #value is opaque to tool, optional
    'target_link_uri': 'https://ltibc.dummy/deeplink' #actual URL to launch at the end of the auth flow
}

# client_id and deployment_id may also be included in the init_login_request, which may be handy to route
# to the right redirect_uri

orig_state = 'XSRF_session_bound_value_{r}'.format(r=randint(0, 100000))

# Using the same value for the nonce
nonce = orig_state

Now that we have the platform's request data and a `state` we can send an auth request, the response will contain the `state` back and the `id_token` which, in addition to carrying the user's identity, contains the information such as course and user's role required to show the proper UI that will allow the user to either pick or create resources to add the course.

The launch flow involves a request from the browser to the platform, which bounces back to the tool. To allow this in this notebook, the actual interaction takes place in an IFrame.

First we create the auth request URL by appending the required parameters to the auth URL provided by us during registration:


In [22]:
auth_request_params = {
    'scope': 'open_id', #we only use the flow for OIDC, not to get an access token
    'response_type': 'id_token',
    'client_id': tool_info['client_id'],
    'login_hint': init_login_request['login_hint'],
    'lti_message_hint': init_login_request['lti_message_hint'],
    'response_mode': 'form_post', # with LTI we expect the user to already have a session in the platform
    'nonce': nonce, #a nonce compared to state should fail if replayed
    'state': orig_state, #state is optional
    'redirect_uri': redirect_uri
}

autopost_iframe('deeplinking', tool_info['oidc_auth_endpoint'], auth_request_params)

<IPython.core.display.Javascript object>

In [23]:
# Set the id_token variable here
# id_token='xxxx'

id_token='eyJraWQiOiIxNTU3NjAzNTgxXzMiLCJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJuYW1lIjoiVGVyZWxsIFN1eiIsImV4cCI6MTU1NzYwNjEyMCwiZmFtaWx5X25hbWUiOiJTdXoiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9kZXBsb3ltZW50X2lkIjoiZGVwbG95bWVudF8xIiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vbHRpMXAxIjp7Im9hdXRoX2NvbnN1bWVyX2tleSI6ImUyNTczLTM3MTg3OSIsIm9hdXRoX2NvbnN1bWVyX2tleV9zaWduIjoiOWFpTmppQUFiVFg1UGk5Q3phdXJnZU5Na0l0czlPV0I3enoxQkFBU0hRVT0ifSwibm9uY2UiOiJYU1JGX3Nlc3Npb25fYm91bmRfdmFsdWVfMzMxNjQiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9sYXVuY2hfcHJlc2VudGF0aW9uIjp7InJldHVybl91cmwiOiJodHRwOi8vbHRpcGxhdGZvcm0udGhlZWR0ZWNoLmRldi90b29sL2ZmNGU4YmM2LTc0MjgtMTFlOS04ZjkwLWZhMTYzZTZiZTY2OS9kbHIiLCJkb2N1bWVudF90YXJnZXQiOiJpZnJhbWUifSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vcm9sZXMiOlsiaHR0cDovL3B1cmwuaW1zZ2xvYmFsLm9yZy92b2NhYi9saXMvdjIvbWVtYmVyc2hpcCNJbnN0cnVjdG9yIl0sInN1YiI6IkxUSUJDVV83IiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vbWVzc2FnZV90eXBlIjoiTFRJRGVlcExpbmtpbmdSZXF1ZXN0IiwiaXNzIjoiaHR0cDovL2x0aXBsYXRmb3JtLnRoZWVkdGVjaC5kZXYiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9jb250ZXh0Ijp7ImlkIjoiZmY0ZThiYzYtNzQyOC0xMWU5LThmOTAtZmExNjNlNmJlNjY5IiwibGFiZWwiOiJMVEkgQm9vdGNhbXAgQ291cnNlIiwidHlwZSI6WyJDb3Vyc2VTZWN0aW9uIl0sInRpdGxlIjoiTFRJIEJvb3RjYW1wIENvdXJzZSJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS1ucnBzL2NsYWltL25hbWVzcm9sZXNlcnZpY2UiOnsiY29udGV4dF9tZW1iZXJzaGlwc191cmwiOiJodHRwOi8vbHRpcGxhdGZvcm0udGhlZWR0ZWNoLmRldi9mZjRlOGJjNi03NDI4LTExZTktOGY5MC1mYTE2M2U2YmU2NjkvbWVtYmVyc2hpcHMifSwiZW1haWwiOiJUZXJlbGwuU3V6QGV4YW1wbGUuY29tIiwiYXVkIjoiMSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWRsL2NsYWltL2RlZXBfbGlua2luZ19zZXR0aW5ncyI6eyJhY2NlcHRfcHJlc2VudGF0aW9uX2RvY3VtZW50X3RhcmdldHMiOlsiaWZyYW1lIiwid2luZG93Il0sImF1dG9fY3JlYXRlIjp0cnVlLCJkYXRhIjoib3A9MzIxJnY9NDQiLCJhY2NlcHRfdHlwZXMiOlsibHRpTGluayJdLCJhY2NlcHRfbXVsdGlwbGUiOnRydWV9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS1hZ3MvY2xhaW0vZW5kcG9pbnQiOnsibGluZWl0ZW1zIjoiaHR0cDovL2x0aXBsYXRmb3JtLnRoZWVkdGVjaC5kZXYvZmY0ZThiYzYtNzQyOC0xMWU5LThmOTAtZmExNjNlNmJlNjY5L2xpbmVpdGVtcyIsInNjb3BlIjpbImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9saW5laXRlbSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9zY29yZSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9yZXN1bHQucmVhZG9ubHkiXX0sImlhdCI6MTU1NzYwNTUyMCwiZ2l2ZW5fbmFtZSI6IlRlcmVsbCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3Rvb2xfcGxhdGZvcm0iOnsibmFtZSI6IkxUSSBCb290Y2FtcCBQbGF0Zm9ybSIsImd1aWQiOiJsdGliY19hdF8xNTU3NjAzNTgyIn0sImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3ZlcnNpb24iOiIxLjMuMCJ9.CzUd3yOW_AyJIQKegf1tOKAPv1gm_DfsoNXoSj28Vf47U-BX1KXQIjXooQcygswvimi60PQlHzf5PmKpe9yqzYCU9joVig8cTZm_DuNI7Wl35n1haeP_ufM3NDB6b3MoLD969DgwkaEU-FZeiOlahyDNynVCHE9FjQ-cdLYTJTHuEhXXMpjaxgXlRUK3RI43-b7TmLfQIfc-fuLhpbEnJPsK2kOrfUgUYEEQySidcJLN11eQbwY6IVBGJ5Vg9P3Gnt64nTol1HOQoZn0wLoff3mG_9itlaTbFTia824Z0XjHEBiN0RrYzP53RSNKdCauVs8pM7BJaa1qgdP8YybEDg'

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [24]:
# Let's get the kid so we can get the proper public key

# should check ISS first to lookup keyset

encoded_jwt = id_token
jwt_headers = jwt.get_unverified_header(encoded_jwt)

md('This request is signed with key: {0}'.format(jwt_headers['kid']))
content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')

if content_item_message['nonce']==nonce:
    md('Nonce match, the response is intended for this notebook' )
else:
    md('Nonce mismatch, this id_token was not requested by this notebook')
    


This request is signed with key: 1557603581_3

#### Message properly signed! Decoded ContentItemSelectionRequest message:

```json
{
    "name": "Terell Suz",
    "exp": 1557606120,
    "family_name": "Suz",
    "https://purl.imsglobal.org/spec/lti/claim/deployment_id": "deployment_1",
    "https://purl.imsglobal.org/spec/lti/claim/lti1p1": {
        "oauth_consumer_key": "e2573-371879",
        "oauth_consumer_key_sign": "9aiNjiAAbTX5Pi9CzaurgeNMkIts9OWB7zz1BAASHQU="
    },
    "nonce": "XSRF_session_bound_value_33164",
    "https://purl.imsglobal.org/spec/lti/claim/launch_presentation": {
        "return_url": "http://ltiplatform.theedtech.dev/tool/ff4e8bc6-7428-11e9-8f90-fa163e6be669/dlr",
        "document_target": "iframe"
    },
    "https://purl.imsglobal.org/spec/lti/claim/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
    ],
    "sub": "LTIBCU_7",
    "https://purl.imsglobal.org/spec/lti/claim/message_type": "LTIDeepLinkingRequest",
    "iss": "http://ltiplatform.theedtech.dev",
    "https://purl.imsglobal.org/spec/lti/claim/context": {
        "id": "ff4e8bc6-7428-11e9-8f90-fa163e6be669",
        "label": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ],
        "title": "LTI Bootcamp Course"
    },
    "https://purl.imsglobal.org/spec/lti-nrps/claim/namesroleservice": {
        "context_memberships_url": "http://ltiplatform.theedtech.dev/ff4e8bc6-7428-11e9-8f90-fa163e6be669/memberships"
    },
    "email": "Terell.Suz@example.com",
    "aud": "1",
    "https://purl.imsglobal.org/spec/lti-dl/claim/deep_linking_settings": {
        "accept_presentation_document_targets": [
            "iframe",
            "window"
        ],
        "auto_create": true,
        "data": "op=321&v=44",
        "accept_types": [
            "ltiLink"
        ],
        "accept_multiple": true
    },
    "https://purl.imsglobal.org/spec/lti-ags/claim/endpoint": {
        "lineitems": "http://ltiplatform.theedtech.dev/ff4e8bc6-7428-11e9-8f90-fa163e6be669/lineitems",
        "scope": [
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem",
            "https://purl.imsglobal.org/spec/lti-ags/scope/score",
            "https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly"
        ]
    },
    "iat": 1557605520,
    "given_name": "Terell",
    "https://purl.imsglobal.org/spec/lti/claim/tool_platform": {
        "name": "LTI Bootcamp Platform",
        "guid": "ltibc_at_1557603582"
    },
    "https://purl.imsglobal.org/spec/lti/claim/version": "1.3.0"
}```

Nonce match, the response is intended for this notebook

### Task 2: extract the information needed to render the selector/authoring UI

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [25]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_settings', 'dl')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
deep_linking_return_url = content_item_message[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(deep_linking_return_url))
mdb('1. Is there any data I must pass back to platform when I return? {0}'.format('data' in deep_linking_claim))
mdb()


1. What is the current course id? ff4e8bc6-7428-11e9-8f90-fa163e6be669
1. What is the current user id? LTIBCU_7
1. Is this user an instructor? True
1. What kind of content item can be created? ['ltiLink']
1. Can I return more than one items to be added? True
1. Will the user be prompted before to actually save the items? False
1. Where should I redirect the browser too when done? http://ltiplatform.theedtech.dev/tool/ff4e8bc6-7428-11e9-8f90-fa163e6be669/dlr
1. Is there any data I must pass back to platform when I return? True

### Task 3: building the response token

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded (since the request supports multiple content items).

Here we're creating the actual response token.

In [26]:
## First let's create our 2 content items
## Note that the URLs are phony as for now there is now way to launch in the notebook anyway...
simple_link =  {
      "type": "ltilink",
      "url": "https://lti.bootcamp/item111",
      "window": {
        "targetName": "bootcamp-win"
      },
      "iframe": {
        "height": 890
      },
      "title": "A simple content item",
      "text": "Some long text",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim4e"
      }
}

assignment_link =  {
      "type": "ltilink",
      "title": "An assignment",
      "url": "https://lti.bootcamp/assignment1",
      "text": "Chemical lab sim",
      "window": {
        "targetName": "examplePublisherContent"
      },
      "iframe": {
        "height": 890
      },
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3a",
        "level": "easy"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      }
}

now = int(time())

deep_linking_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    fc('message_type'): "DeepLinkingResponse",
    fc('version'): "1.3.0",
    fc('content_items', 'dl'): [
        simple_link, assignment_link
    ]
    
}

### Task 4: build the signed JWT

In [27]:
deep_linking_response_token = jwt.encode(deep_linking_response, notebook_private_key, 'RS256').decode()

print(deep_linking_response_token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJpc3MiOiIxIiwiYXVkIjoiaHR0cDovL2x0aXBsYXRmb3JtLnRoZWVkdGVjaC5kZXYiLCJleHAiOjE1NTc2MDU2MjQsImlhdCI6MTU1NzYwNTU2NCwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vbWVzc2FnZV90eXBlIjoiRGVlcExpbmtpbmdSZXNwb25zZSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3ZlcnNpb24iOiIxLjMuMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWRsL2NsYWltL2NvbnRlbnRfaXRlbXMiOlt7InR5cGUiOiJsdGlsaW5rIiwidXJsIjoiaHR0cHM6Ly9sdGkuYm9vdGNhbXAvaXRlbTExMSIsIndpbmRvdyI6eyJ0YXJnZXROYW1lIjoiYm9vdGNhbXAtd2luIn0sImlmcmFtZSI6eyJoZWlnaHQiOjg5MH0sInRpdGxlIjoiQSBzaW1wbGUgY29udGVudCBpdGVtIiwidGV4dCI6IlNvbWUgbG9uZyB0ZXh0IiwiaWNvbiI6eyJ1cmwiOiJodHRwOi8vbHRpLmV4YW1wbGUuY29tL2ltYWdlLmpwZyIsIndpZHRoIjoxMDAsImhlaWdodCI6MTAwfSwiY3VzdG9tIjp7ImxhYiI6InNpbTRlIn19LHsidHlwZSI6Imx0aWxpbmsiLCJ0aXRsZSI6IkFuIGFzc2lnbm1lbnQiLCJ1cmwiOiJodHRwczovL2x0aS5ib290Y2FtcC9hc3NpZ25tZW50MSIsInRleHQiOiJDaGVtaWNhbCBsYWIgc2ltIiwid2luZG93Ijp7InRhcmdldE5hbWUiOiJleGFtcGxlUHVibGlzaGVyQ29udGVudCJ9LCJ

### Task 5: redirect the user back to the platform with the content item selection

Now that we the response token, let's do the actual HTML POST redirection to the platform. Note that because the platform supports `autocreate` there will be no prompt. The 2 items will be added directly to the course.


In [28]:

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame');
                $('<input>').attr({{
                    type: 'hidden',
                    name: 'jws_token',
                    value: '{token}'
                }}).appendTo(ltiForm);
                $('#deeplinking_frame').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

autosubmit_js = autosubmit_js.format(url=deep_linking_return_url, token=deep_linking_response_token)

display(HTML('<iframe id="deeplinking_frame" name="deeplinking_frame" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

<IPython.core.display.Javascript object>

## Student Resource Link launch

Now that we have created resource links, let's handle a student launch from one of them. We're going to use a resource link with a **coupled** line item, so that we can use it to send a score back to the platform.

### Setup

The first thing we need, as with deep linking, is to get from the test platform the launch token which an actual tool would get in an actual HTML Form Post.

In [48]:
# all messages are sent the same way, the Learning Platform indicates it wants to start an auth flow;
# login_hint, lti_message_hint if used are black box to the tool and must be passed untouched in the
# auth request

# in our case though we will build here the lti_message_hint needed by the Platform

context_id= content_item_message[fc('context')]['id']

# we want to launch in the assignment
# select the assignment id from the ones displayed in the course platform in the IFrame above
resource_link_id='d74a7af9-40f3-11e9-b6a6-024283bd1de6' 

message_hint = '{0}rlid{1}'.format(context_id, resource_link_id)

init_login_request = {
    'iss': platform_url,
    'login_hint': 'student', #value is opaque to tool, ex: tie to end user's platform's session
    'lti_message_hint': message_hint,
    'target_link_uri': 'https://lti.bootcamp/assignment1' #actual URL to launch at the end of the flow, would be the url provided in LtiResourceLink payload if provided
}

orig_state = 'XSRF_session_bound_value_{r}'.format(r=randint(0, 100000))

# Using the same value for the nonce
nonce = orig_state

auth_request_params = {
    'scope': 'open_id', #we only use the flow for OIDC, not to get an access token
    'response_type': 'id_token',
    'client_id': tool_info['client_id'],
    'login_hint': init_login_request['login_hint'],
    'lti_message_hint': init_login_request['lti_message_hint'],
    'response_mode': 'form_post', # with LTI we expect the user to already have a session in the platform
    'nonce': nonce, #a nonce compared to state should fail if replayed
    'state': orig_state, #state is optional
    'redirect_uri': redirect_uri
}

autopost_iframe('resourcelink', tool_info['oidc_auth_endpoint'], auth_request_params)

<IPython.core.display.Javascript object>

### Task 1: Decode the launch

Now, same as with the Deep Linking request, we decode the token:

In [49]:
# paste token here id_token = ''
id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IjE1NTE5NzUxMDNfMSJ9.eyJub25jZSI6IlhTUkZfc2Vzc2lvbl9ib3VuZF92YWx1ZV82MjgxNiIsImlhdCI6MTU1MTk3NjQwOSwiZXhwIjoxNTUxOTc3MDA5LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAiLCJhdWQiOiIwIiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vZGVwbG95bWVudF9pZCI6ImRlcGxveW1lbnRfMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL21lc3NhZ2VfdHlwZSI6IkxUSVJlc291cmNlTGlua0xhdW5jaCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3ZlcnNpb24iOiIxLjMuMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL2xhdW5jaF9wcmVzZW50YXRpb24iOnsiZG9jdW1lbnRfdGFyZ2V0IjoiaWZyYW1lIiwicmV0dXJuX3VybCI6Imh0dHA6Ly9sb2NhbGhvc3Q6NTAwMGh0dHA6Ly9sb2NhbGhvc3Q6NTAwMC8ifSwic3ViIjoiTFRJQkNVXzEwIiwiZ2l2ZW5fbmFtZSI6IkViYmEiLCJmYW1pbHlfbmFtZSI6IkVsaXNhIiwibmFtZSI6IkViYmEgRWxpc2EiLCJlbWFpbCI6IkViYmEuRWxpc2FAZXhhbXBsZS5jb20iLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9yb2xlcyI6WyJodHRwOi8vcHVybC5pbXNnbG9iYWwub3JnL3ZvY2FiL2xpcy92Mi9tZW1iZXJzaGlwI0xlYXJuZXIiXSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vY29udGV4dCI6eyJpZCI6ImJiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNiIsImxhYmVsIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInRpdGxlIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInR5cGUiOlsiQ291cnNlU2VjdGlvbiJdfSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vcmVzb3VyY2VfbGluayI6eyJpZCI6ImQ3NGE3YWY5LTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNiIsInRpdGxlIjoiQW4gYXNzaWdubWVudCJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9jdXN0b20iOnsibGFiIjoic2ltM2EiLCJsZXZlbCI6ImVhc3kifSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktYWdzL2NsYWltL2VuZHBvaW50Ijp7InNjb3BlIjpbImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9saW5laXRlbSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9zY29yZSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9yZXN1bHQucmVhZG9ubHkiXSwibGluZWl0ZW1zIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2JiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNi9saW5laXRlbXMiLCJsaW5laXRlbSI6Imh0dHA6Ly9sb2NhbGhvc3Q6NTAwMC9iYmI5YWJlYy00MGYzLTExZTktYjZhNi0wMjQyODNiZDFkZTYvbGluZWl0ZW1zLzEvbGluZWl0ZW0ifSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktbnJwcy9jbGFpbS9uYW1lc3JvbGVzZXJ2aWNlIjp7ImNvbnRleHRfbWVtYmVyc2hpcHNfdXJsIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2JiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNi9tZW1iZXJzaGlwcyJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS90b29sX3BsYXRmb3JtIjp7Im5hbWUiOiJMVEkgQm9vdGNhbXAgUGxhdGZvcm0iLCJndWlkIjoibHRpYmNfYXRfMTU1MTk3NTEwNSJ9fQ.a5Ebs-gi4SL2jPMoByWKzchImZHumf_7s3_h8iht5hWNz4ZtbV1FzW4PaNoF_lucgA4mFKwwQykLjpgSazXfn4VBTcKHjJmqoZSIXH5CtO0OCUTn2Qem0jJv7vdta4cQAN661FlnkR5fH103QRqd4vLa3Ny0ltn25V2YEb35HLEVR26_NiF6TdI1FSxDnqQrRGI8jlXt8obJ_sKFYkFTWWw70jD0-O_dU_gPYbVorETLDYYvr9naIRolwXDoNN11ozaGCv4H56G2_m4-ycEmnGKfAlVQId8VGJAPSerS0hKAVU3P2uTIP5X1eW5E3oop3P1ZFq9l99ZSRH-_JE_8ZA'
jwt_headers = jwt.get_unverified_header(id_token)

student_launch = jwt.decode(id_token, 
                            platform_keys[jwt_headers['kid']], 
                            jwt_headers['alg'],
                            audience = tool_info['client_id'])

md('```json\n'+ json.dumps(student_launch, indent=4)+'```')

```json
{
    "nonce": "XSRF_session_bound_value_62816",
    "iat": 1551976409,
    "exp": 1551977009,
    "iss": "http://localhost:5000",
    "aud": "0",
    "https://purl.imsglobal.org/spec/lti/claim/deployment_id": "deployment_0",
    "https://purl.imsglobal.org/spec/lti/claim/message_type": "LTIResourceLinkLaunch",
    "https://purl.imsglobal.org/spec/lti/claim/version": "1.3.0",
    "https://purl.imsglobal.org/spec/lti/claim/launch_presentation": {
        "document_target": "iframe",
        "return_url": "http://localhost:5000http://localhost:5000/"
    },
    "sub": "LTIBCU_10",
    "given_name": "Ebba",
    "family_name": "Elisa",
    "name": "Ebba Elisa",
    "email": "Ebba.Elisa@example.com",
    "https://purl.imsglobal.org/spec/lti/claim/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
    ],
    "https://purl.imsglobal.org/spec/lti/claim/context": {
        "id": "bbb9abec-40f3-11e9-b6a6-024283bd1de6",
        "label": "LTI Bootcamp Course",
        "title": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ]
    },
    "https://purl.imsglobal.org/spec/lti/claim/resource_link": {
        "id": "d74a7af9-40f3-11e9-b6a6-024283bd1de6",
        "title": "An assignment"
    },
    "https://purl.imsglobal.org/spec/lti/claim/custom": {
        "lab": "sim3a",
        "level": "easy"
    },
    "https://purl.imsglobal.org/spec/lti-ags/claim/endpoint": {
        "scope": [
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem",
            "https://purl.imsglobal.org/spec/lti-ags/scope/score",
            "https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly"
        ],
        "lineitems": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems",
        "lineitem": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems/1/lineitem"
    },
    "https://purl.imsglobal.org/spec/lti-nrps/claim/namesroleservice": {
        "context_memberships_url": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships"
    },
    "https://purl.imsglobal.org/spec/lti/claim/tool_platform": {
        "name": "LTI Bootcamp Platform",
        "guid": "ltibc_at_1551975105"
    }
}```

### Task 2: extract information to show the correct activity

The launch gives information about the user, her role, the course, but also which actual resource we want to launch into.

In [50]:
# fc(claim) prefix the claim with http://imsglobal.org/lti/
mdb('1. Is this a resource link launch? {0}'.format(student_launch[fc('message_type')] == 'LTIResourceLinkLaunch'))
mdb('1. What is the id of the resource link that is launched? {0}'.format(student_launch[fc('resource_link')]['id']))
mdb('1. What is the name of the resource that is launched? {0}'.format(student_launch[fc('resource_link')]['title']))
mdb('1. What is the current course id? {0}'.format(student_launch[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(student_launch['sub']))
is_learner = len(list(filter(lambda x: 'learner' in x.lower(), student_launch[fc('roles')])))>0
mdb('1. Is this user a student? {0}'.format(is_learner))
return_url = student_launch[fc('launch_presentation')]['return_url']
mdb('1. Where should I redirect the browser too when done? {0}'.format(return_url))
mdb('1. Which lab should be displayed? {0}'.format(student_launch[fc('custom')]['lab']))
ags_claim = student_launch[fc('endpoint', 'ags')]
mdb('1. Is there a gradebook column for this resource? {0}'.format('lineitem' in ags_claim))
mdb()



1. Is this a resource link launch? True
1. What is the id of the resource link that is launched? d74a7af9-40f3-11e9-b6a6-024283bd1de6
1. What is the name of the resource that is launched? An assignment
1. What is the current course id? bbb9abec-40f3-11e9-b6a6-024283bd1de6
1. What is the current user id? LTIBCU_10
1. Is this user a student? True
1. Where should I redirect the browser too when done? http://localhost:5000http://localhost:5000/
1. Which lab should be displayed? sim3a
1. Is there a gradebook column for this resource? True

## Assignment and Grade Services: Posting a grade to a coupled line item

Now that the student has launched into a grading activity, eventually she will complete it. Let's assume this is an autograded quiz. At the end of the interaction, we're going to send a score.

<img src="assets/ags.png" width="60%">

The line item (gradebook column) was created declaratively by the platform when the resource link was created (at the end of the deep linking flow). The line item and the resource link are **coupled**. Due to this explicit relationship, the platform passes us directly the associated line item URL, which we will use to send scores. 

### Task 1: Get an access token

To be able to send a grade, or call any service on that matter, we must first get an access token. This is done by using a JWT based client grant flow [RFC-7523](https://tools.ietf.org/html/rfc7523).

Here we will re-use the token for the rest of the notebook, so we don't specify scope. If you intend to use the token only for a given operation, it is a good practice to scope it accordingly.

The grant type is [client_credentials](https://tools.ietf.org/html/rfc6749#section-1.3.4) as the trust is established between the tool and the platform. The current user and context are not considered.


In [56]:
## Let's define auth functions we can re-use for other calls

private_key_pem = notebook_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption())


def get_token(scope):
    display(scope)
    token_endpoint = tool_info['accesstoken_endpoint']

    now = int(time())

    assertion = {
        "iss": tool_info['client_id'],
        "aud": token_endpoint,
        "exp": now + 60,
        "iat": now,
        "jti": "{0}-{1}".format(tool_info['client_id'], now)
    }

    assertion_jwt = jwt.encode(assertion, private_key_pem, 'RS256').decode()

    return json.loads(requests.post(token_endpoint, data = {
        'client_assertion': assertion_jwt,
        'grant_type': 'client_credentials',
        'scope': scope,
        'client_assertion_type': 'urn:ietf:params:oauth:client-assertion-type:jwt-bearer'
    }).text)

# We'll also need to create a proper header, so let's also create a function for that
def add_authorization(headers, access_token):
    b64token = base64.b64encode(access_token.encode('utf-8')).decode()
    headers.update({'Authorization': 'Bearer {0}'.format(b64token)})

    


In [57]:
token_info = get_token('{0} {1}'.format(scope('result.readonly', s='ags'), scope('score', s='ags')))
md('#### Access token:')
md('```json\n'+ json.dumps(token_info, indent=4)+'```')

'https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly https://purl.imsglobal.org/spec/lti-ags/scope/score'

#### Access token:

```json
{
    "access_token": "tk0d24b990-40f9-11e9-b6a6-024283bd1de6",
    "expires_in": 3600,
    "token_type": "Bearer"
}```

### Task 2: Publish a score

In [58]:
# Scores in the subpath scores from lineitem.
def append_to_path(path, subpath):
    p = re.compile('lineitem($|\?|#)')
    return p.sub('lineitem/{0}\\1'.format(subpath), path)

scores_url = append_to_path(ags_claim['lineitem'], 'scores')

score = {
    'userId': student_launch['sub'],
    'scoreGiven': 9,
    'scoreMaximum': 10,
    'activityProgress': 'Completed',
    'gradingProgress': 'FullyGraded',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score), verify=False)

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')




The score was processed successfully be the back-end

### Task 3: get the results

Let's not get the results to see our operation did actually succeed

In [59]:
results_url = append_to_path(ags_claim['lineitem'], 'results')

headers = {'Content-Type': 'application/vnd.ims.lis.v2.resultcontainer+json'}
add_authorization(headers, token_info['access_token'])

r = requests.get(results_url.encode(), headers=headers)

# let's check it was OK
r.raise_for_status()

md('#### Current results for item')
md('```json\n'+ json.dumps(r.json(), indent=4)+'```')



#### Current results for item

```json
[
    {
        "resultMaximum": 34,
        "resultScore": 30.6,
        "userId": "LTIBCU_10"
    }
]```

### Task 4: marking a score as needing grading
Assignment and Grade services allow for more than just passing grade. Let's say we just want to mark a submission was done, and the instructor requires to manually grade the essay. In which case, we can post a Score indicating the status of the activity is completed, and the grading needs manual intervention using the `activityProgress` and `gradingProgress` attributes.

In [60]:
score = {
    'userId': student_launch['sub'],
    'activityProgress': 'Completed',
    'gradingProgress': 'PendingManual',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score))

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')


The score was processed successfully be the back-end

We can see it now in the platform gradebook:

In [61]:
platform_gradebook_url = '{0}/course/{1}/gradebook'.format(platform_url, student_launch[fc('context')]['id'])
display(HTML('<iframe src="{0}" style="height: 300px; width:100%"></iframe>'.format(platform_gradebook_url)))

## Assignment and Grade Services - Uncoupled

The assignment and grade services support an alternate way to create line items: rather than having the platform creates a line item when it creates a link, a tool may use the API to create standalone line items, i.e. uncoupled from any lti link (the API may optionaly include an ltiLinkId to tie the one or many line items to a single resource link, although platforms will often treat this association as weaker than a declarative flow).

Let's use the API to add a new line item, send some scores to it, then delete it.


### Task 1: get a properly scoped token

Same as before, we just need to now ask also for the scope to allow line item management.

In [62]:
token_info = get_token(' '.join([scope('lineitem', s='ags'), scope('score', s='ags'), scope('result.readonly', 'ags')]))

md('#### Access token with all assignment and grade services scopes:')
md('```json\n'+ json.dumps(token_info, indent=4)+'```')

'https://purl.imsglobal.org/spec/lti-ags/scope/lineitem https://purl.imsglobal.org/spec/lti-ags/scope/score https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly'

#### Access token with all assignment and grade services scopes:

```json
{
    "access_token": "tk20985036-40f9-11e9-b6a6-024283bd1de6",
    "expires_in": 3600,
    "token_type": "Bearer"
}```

### Task 2: create a new line item and post score

The `lineitems` is the URL representating all the line items in the current context bound to our tool. We can do a GET to see all the existing items, and a POST to add a new one. Let's start by adding one:

In [63]:
# note the specific mediatype
headers = {'Content-Type': 'application/vnd.ims.lis.v2.lineitem+json'}
add_authorization(headers, token_info['access_token'])

lineitem = {
    "resourceId": "bootcamp",
    "tag": "participation",
    "label": "Participation in Bootcamp",
    "scoreMaximum": 25
}

r = requests.post(ags_claim['lineitems'], headers=headers, data=json.dumps(lineitem), verify=False)

r.raise_for_status()

uncoupled_item = r.json();

md('#### Successfully added line item:')
md('```json\n'+ json.dumps(uncoupled_item, indent=4)+'```')



#### Successfully added line item:

```json
{
    "id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems/2/lineitem",
    "label": "Participation in Bootcamp",
    "resourceId": "bootcamp",
    "scoreMaximum": 25,
    "tag": "participation"
}```

In [64]:
r = requests.get(ags_claim['lineitems'], headers=headers, verify=False)

# let's check it was OK
r.raise_for_status()

md('#### Current line items for this tool in the context')
md('```json\n'+ json.dumps(r.json(), indent=4)+'```')

#### Current line items for this tool in the context

```json
[
    {
        "id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems/1/lineitem",
        "label": "Chemical lab sim",
        "ltiLinkId": "d74a7af9-40f3-11e9-b6a6-024283bd1de6",
        "resourceId": "sim3a",
        "scoreMaximum": 34,
        "tag": "final_grade"
    },
    {
        "id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems/2/lineitem",
        "label": "Participation in Bootcamp",
        "resourceId": "bootcamp",
        "scoreMaximum": 25,
        "tag": "participation"
    }
]```

Sending the score to the uncoupled line item is the same as with the coupled one, the id being the line item URL.

In [65]:
scores_url = append_to_path(uncoupled_item['id'], 'scores')

score = {
    'userId': student_launch['sub'],
    'scoreGiven': 10,
    'scoreMaximum': 10,
    'activityProgress': 'Completed',
    'gradingProgress': 'FullyGraded',
    'timestamp': datetime.utcnow().isoformat()
}

headers = {'Content-Type': 'application/vnd.ims.lis.v1.score+json'}
add_authorization(headers, token_info['access_token'])

r = requests.post(scores_url.encode(), headers=headers, data=json.dumps(score), verify=False)

# let's check it was OK
r.raise_for_status()

md('The score was processed successfully be the back-end')


The score was processed successfully be the back-end

### Task 3: Updating the line item

Update is done by PUTting the updated definition to the line item  URL (id).

In [66]:
# note the specific mediatype
headers = {'Content-Type': 'application/vnd.ims.lis.v2.lineitem+json'}
add_authorization(headers, token_info['access_token'])

lineitem = {
    "resourceId": "bootcamp",
    "tag": "participation",
    "label": "Participation (uncoupled)",
    "scoreMaximum": 30
}

r = requests.put(uncoupled_item['id'], headers=headers, data=json.dumps(lineitem), verify=False)
                           
r.raise_for_status()

uncoupled_item = r.json();

md('#### Successfully updated line item:')
md('```json\n'+ json.dumps(uncoupled_item, indent=4)+'```')


#### Successfully updated line item:

```json
{
    "id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/lineitems/2/lineitem",
    "label": "Participation (uncoupled)",
    "scoreMaximum": 30,
    "tag": "participation"
}```

You can also scroll back up where the platform UI is displayed and check in the gradebook to see the item is present (and scores have been rescaled).

### Task 4: Delete the uncoupled line item

Deleting is just issuing a DELETE command the line item URL.

In [67]:
# no content type, it's a delete
headers = {}
add_authorization(headers, token_info['access_token'])

r = requests.delete(uncoupled_item['id'], headers=headers, verify=False)
                           
r.raise_for_status()

md('#### Successfully deleted line item')


#### Successfully deleted line item

## Names and Role Provisioning Service

Or more simply, membership service, lets you get the roster of the course.

<img src="assets/membership.png" width="60%">

This service does not allow any modification of the course roster. At a minimum, the platform will apply the same privacy restrictions that applies to actual launches.

### Task 1: Get the full course roster

The Assignment and Grade Service endpoint is communicated in the LTI message under the claim `https://purl.imsglobal.org/lti/claim/arps`. The scope to request is `https://purl.imsglobal.org/lti/scope/nrps`.



In [68]:
## Get An Access Token

membership_token = get_token(scope('contextmembership.readonly', s='nrps'))

## Request the enrollment

headers = {'Content-Type': 'application/vnd.ims.lis.v2.membershipcontainer+json'}
add_authorization(headers, membership_token['access_token'])
membership_url = student_launch[fc('namesroleservice', s='nrps')]['context_memberships_url']

r = requests.get(membership_url.encode(), headers=headers, verify=False)

r.raise_for_status()

roster = r.json();

md('#### Successfully retrieved course roster:')
md('```json\n'+ json.dumps(roster, indent=4)+'```')


'https://purl.imsglobal.org/spec/lti-nrps/scope/contextmembership.readonly'

#### Successfully retrieved course roster:

```json
{
    "@context": [
        "http://purl.imsglobal.org/ctx/lis/v2/MembershipContainer",
        {
            "lism": "http://purl.imsglobal.org/vocab/lis/v2/membership#",
            "liss": "http://purl.imsglobal.org/vocab/lis/v2/status#"
        }
    ],
    "@id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships",
    "@type": "Page",
    "differences": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships?since=1551977445",
    "pageOf": {
        "@type": "LISMembershipContainer",
        "membershipSubject": {
            "@type": "Context",
            "contextId": "bbb9abec-40f3-11e9-b6a6-024283bd1de6",
            "membership": [
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Fleurette.Michel@example.com",
                        "familyName": "Michel",
                        "givenName": "Fleurette",
                        "name": "Fleurette Michel",
                        "sourcedId": "b3d662a0-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_2"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Kara.Thrace@example.com",
                        "familyName": "Thrace",
                        "givenName": "Kara",
                        "name": "Kara Thrace",
                        "sourcedId": "b3d662ab-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_13"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership/instructor#TeachingAssistant"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Lucia.Colette@example.com",
                        "familyName": "Colette",
                        "givenName": "Lucia",
                        "name": "Lucia Colette",
                        "sourcedId": "b3d662a2-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_4"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Sander.Donald@example.com",
                        "familyName": "Donald",
                        "givenName": "Sander",
                        "name": "Sander Donald",
                        "sourcedId": "b3d662a4-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_6"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Horatio.Maya@example.com",
                        "familyName": "Maya",
                        "givenName": "Horatio",
                        "name": "Horatio Maya",
                        "sourcedId": "b3d662a7-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_9"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Colby.Rona@example.com",
                        "familyName": "Rona",
                        "givenName": "Colby",
                        "name": "Colby Rona",
                        "sourcedId": "b3d662a3-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_5"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "William.Adama@example.com",
                        "familyName": "Adama",
                        "givenName": "William",
                        "name": "William Adama",
                        "sourcedId": "b3d662ac-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_14"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Shouta.Akio@example.com",
                        "familyName": "Akio",
                        "givenName": "Shouta",
                        "name": "Shouta Akio",
                        "sourcedId": "b3d662b1-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_19"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Christal.Conch\u00far@example.com",
                        "familyName": "Conch\u00far",
                        "givenName": "Christal",
                        "name": "Christal Conch\u00far",
                        "sourcedId": "b3d662a1-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_3"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Ebba.Elisa@example.com",
                        "familyName": "Elisa",
                        "givenName": "Ebba",
                        "name": "Ebba Elisa",
                        "sourcedId": "b3d662a8-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_10"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Saul.Tigh@example.com",
                        "familyName": "Tigh",
                        "givenName": "Saul",
                        "name": "Saul Tigh",
                        "sourcedId": "b3d662af-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_17"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Gaius.Baltar@example.com",
                        "familyName": "Baltar",
                        "givenName": "Gaius",
                        "name": "Gaius Baltar",
                        "sourcedId": "b3d662ad-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_15"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Galen.Tyrol@example.com",
                        "familyName": "Tyrol",
                        "givenName": "Galen",
                        "name": "Galen Tyrol",
                        "sourcedId": "b3d662b0-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_18"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Kyou.Yevheniya@example.com",
                        "familyName": "Yevheniya",
                        "givenName": "Kyou",
                        "name": "Kyou Yevheniya",
                        "sourcedId": "b3d662b3-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_21"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Esther.Raphael@example.com",
                        "familyName": "Raphael",
                        "givenName": "Esther",
                        "name": "Esther Raphael",
                        "sourcedId": "b3d662a6-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_8"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Sharon.Valerii@example.com",
                        "familyName": "Valerii",
                        "givenName": "Sharon",
                        "name": "Sharon Valerii",
                        "sourcedId": "b3d662ae-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_16"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                }
            ]
        }
    }
}```

## Task 2: Get all changes since previous sync

Some platforms will include in their membership payload a `differences` url. This url, when present, allows the tool to update the roster based on the changes since last time the membership was returned, avoiding to transfer the full roster each time.

Use the differences url above to check for updates in roster.



In [69]:
r = requests.get(roster['differences'].encode(), headers=headers, verify=False)

r.raise_for_status()

roster_updates = r.json();

md('#### Successfully retrieved roster update:')
md('```json\n'+ json.dumps(roster_updates, indent=4)+'```')


#### Successfully retrieved roster update:

```json
{
    "@context": [
        "http://purl.imsglobal.org/ctx/lis/v2/MembershipContainer",
        {
            "lism": "http://purl.imsglobal.org/vocab/lis/v2/membership#",
            "liss": "http://purl.imsglobal.org/vocab/lis/v2/status#"
        }
    ],
    "@id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships",
    "@type": "Page",
    "differences": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships?since=1551977448",
    "pageOf": {
        "@type": "LISMembershipContainer",
        "membershipSubject": {
            "@type": "Context",
            "contextId": "bbb9abec-40f3-11e9-b6a6-024283bd1de6",
            "membership": []
        }
    }
}```

## Task 3: get all students and paging

The service allows also to filter by roles using the `role` parameter. Both short and long version of roles can be used to filter (for example `Learner` and `http://purl.imsglobal.org/vocab/lis/v2/membership#Learner`). So let's limit the results to 3 students using the `limit` parameter.

In [70]:
params = {'role':'Learner', 'limit': 3}

r = requests.get(membership_url.encode(), params=params, headers=headers, verify=False)

r.raise_for_status()

roster = r.json();

md('#### Students only:')
md('```json\n'+ json.dumps(roster, indent=4)+'```')

#### Students only:

```json
{
    "@context": [
        "http://purl.imsglobal.org/ctx/lis/v2/MembershipContainer",
        {
            "lism": "http://purl.imsglobal.org/vocab/lis/v2/membership#",
            "liss": "http://purl.imsglobal.org/vocab/lis/v2/status#"
        }
    ],
    "@id": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships",
    "@type": "Page",
    "differences": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships?since=1551977451",
    "nextPage": "http://localhost:5000/bbb9abec-40f3-11e9-b6a6-024283bd1de6/memberships?limit=3&from=3&role=Learner",
    "pageOf": {
        "@type": "LISMembershipContainer",
        "membershipSubject": {
            "@type": "Context",
            "contextId": "bbb9abec-40f3-11e9-b6a6-024283bd1de6",
            "membership": [
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Lucia.Colette@example.com",
                        "familyName": "Colette",
                        "givenName": "Lucia",
                        "name": "Lucia Colette",
                        "sourcedId": "b3d662a2-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_4"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Sander.Donald@example.com",
                        "familyName": "Donald",
                        "givenName": "Sander",
                        "name": "Sander Donald",
                        "sourcedId": "b3d662a4-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_6"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                },
                {
                    "member": {
                        "@type": "LISPerson",
                        "email": "Horatio.Maya@example.com",
                        "familyName": "Maya",
                        "givenName": "Horatio",
                        "name": "Horatio Maya",
                        "sourcedId": "b3d662a7-40f3-11e9-b6a6-024283bd1de6",
                        "userId": "LTIBCU_9"
                    },
                    "role": [
                        "http://purl.imsglobal.org/vocab/lis/v2/membership#Learner"
                    ],
                    "status": "liss:Active"
                }
            ]
        }
    }
}```

# Substitution parameters - tailor your launch

Substitution parameters allow to have custom parameters resolve at runtime information from the platform. For example, if you would like to know the context history, i.e. from which course this course is a copy from, there is the substitution parameter: `$Context.id.history`. Substitution parameters always start will $ . If the platform supports it, the value will be substituted at runtime, otherwise it will remain `$Context.id.history`.

Substitution parameters really allow the tool to tailor its launches based on its particular needs. Substitution parameters may be set during the tool deployment. But it really shines with Deep Linking where the custom parameters can express the substitution parameters to resolve for the added LTI Link.

## Task 1: create an LTI Link with a line item and due date support

In this exercise, we're going to ask the platform to give us the due date of the activity (when a student is last authorized to submit) `$ResourceLink.submission.endDateTime` and the context history `$Context.id.history`. Since we're at it, we're also going to communicate an initial due date time using the `submission.endDateTime` property in the deep linking item.


In [72]:
# Prentending this is what we received from the Platform to initiate the deep linking flow
init_login_request = {
    'iss': platform_url,
    'login_hint': 'instructor', #value is opaque to tool, ex: tie to end user's platform's session
    'lti_message_hint': 'deeplink', #value is opaque to tool, optional
    'target_link_uri': 'https://ltibc.dummy/deeplink' #actual URL to launch at the end of the auth flow
}

orig_state = 'XSRF_session_bound_value_{r}'.format(r=randint(0, 100000))

# Using the same value for the nonce
nonce = orig_state

auth_request_params = {
    'scope': 'open_id', #we only use the flow for OIDC, not to get an access token
    'response_type': 'id_token',
    'client_id': tool_info['client_id'],
    'login_hint': init_login_request['login_hint'],
    'lti_message_hint': init_login_request['lti_message_hint'],
    'response_mode': 'form_post', # with LTI we expect the user to already have a session in the platform
    'nonce': nonce, #a nonce compared to state should fail if replayed
    'state': orig_state, #state is optional
    'redirect_uri': redirect_uri
}

autopost_iframe('deep_linking_sub_params', tool_info['oidc_auth_endpoint'], auth_request_params)

<IPython.core.display.Javascript object>

In [74]:

# paste id_token here
id_token = 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IjE1NTE5NzUxMDNfMiJ9.eyJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS1kbC9jbGFpbS9kZWVwX2xpbmtpbmdfc2V0dGluZ3MiOnsiYWNjZXB0X3R5cGVzIjpbImx0aUxpbmsiXSwiYWNjZXB0X3ByZXNlbnRhdGlvbl9kb2N1bWVudF90YXJnZXRzIjpbImlmcmFtZSIsIndpbmRvdyJdLCJhY2NlcHRfbXVsdGlwbGUiOnRydWUsImF1dG9fY3JlYXRlIjp0cnVlLCJkYXRhIjoib3A9MzIxJnY9NDQifSwibm9uY2UiOiJYU1JGX3Nlc3Npb25fYm91bmRfdmFsdWVfNjkxOTAiLCJpYXQiOjE1NTIyMjUzMDUsImV4cCI6MTU1MjIyNTkwNSwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwIiwiYXVkIjoiMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL2RlcGxveW1lbnRfaWQiOiJkZXBsb3ltZW50XzAiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9tZXNzYWdlX3R5cGUiOiJMVElEZWVwTGlua2luZ1JlcXVlc3QiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS92ZXJzaW9uIjoiMS4zLjAiLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9sYXVuY2hfcHJlc2VudGF0aW9uIjp7ImRvY3VtZW50X3RhcmdldCI6ImlmcmFtZSIsInJldHVybl91cmwiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAvdG9vbC9iYmI5YWJlYy00MGYzLTExZTktYjZhNi0wMjQyODNiZDFkZTYvZGxyIn0sInN1YiI6IkxUSUJDVV8yIiwiZ2l2ZW5fbmFtZSI6IkZsZXVyZXR0ZSIsImZhbWlseV9uYW1lIjoiTWljaGVsIiwibmFtZSI6IkZsZXVyZXR0ZSBNaWNoZWwiLCJlbWFpbCI6IkZsZXVyZXR0ZS5NaWNoZWxAZXhhbXBsZS5jb20iLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9yb2xlcyI6WyJodHRwOi8vcHVybC5pbXNnbG9iYWwub3JnL3ZvY2FiL2xpcy92Mi9tZW1iZXJzaGlwI0luc3RydWN0b3IiXSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vY29udGV4dCI6eyJpZCI6ImJiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNiIsImxhYmVsIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInRpdGxlIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInR5cGUiOlsiQ291cnNlU2VjdGlvbiJdfSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktYWdzL2NsYWltL2VuZHBvaW50Ijp7InNjb3BlIjpbImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9saW5laXRlbSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9zY29yZSIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpLWFncy9zY29wZS9yZXN1bHQucmVhZG9ubHkiXSwibGluZWl0ZW1zIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2JiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNi9saW5laXRlbXMifSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktbnJwcy9jbGFpbS9uYW1lc3JvbGVzZXJ2aWNlIjp7ImNvbnRleHRfbWVtYmVyc2hpcHNfdXJsIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2JiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNi9tZW1iZXJzaGlwcyJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS90b29sX3BsYXRmb3JtIjp7Im5hbWUiOiJMVEkgQm9vdGNhbXAgUGxhdGZvcm0iLCJndWlkIjoibHRpYmNfYXRfMTU1MTk3NTEwNSJ9fQ.NiJ1HzefbfCKKj3FVYQlgeDD154iN-UgXKzhrdVEUJtUAyGJjGK8d3-ePz7Rpi-FFq8O8Wep5GzagjbqL-BRI5jBQxC3VMuqSNV1gTrjfhUZem8fvm_mPwr7yMt3l88-YdVZ2eLnfvJOwi6gPca1EEAcb5rWXYj2-N9qXODbkD8sN9LEzjU_x2ILS-2XtOBIoFPyoDvehbtPn7OMWSQpkSc1x60yLGe3TeAUZH9CzRkMGIR709mpG7AJilam_jJjdgxGHr90CpCoKDxWHtaHFDT1ZZOwTuctiAyU-rNkZ-G04kO-65w2I2kioFDJltTX_72nroyod405Hm-W43BPkA' 
jwt_headers = jwt.get_unverified_header(id_token)

content_item_message = jwt.decode(id_token, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

# Adding an LTI link with a line item and end date

due_date = datetime.utcnow() + timedelta(days=7)

duedate_assignment_link =  {
      "type": "ltilink",
      "url": "http://lti.bootcamp/item111",
      "title": "Assignment with Dates",
      "text": "Do the assignment before the due date",
      "icon": {
        "url": "http://lti.example.com/image.jpg",
        "width": 100,
        "height": 100
      },
      "custom": {
        "lab": "sim3b",
        "dueDate": "$ResourceLink.submission.endDateTime",
        "contextHistory": "$Context.id.history"
      },
      "lineItem": {
        "scoreMaximum": 34,
        "label": "Chemical lab sim",
        "resourceId": "sim3a",
        "tag": "final_grade"
      },
      "submission": {
          "endDateTime": due_date.isoformat()
      },
      "window": {
        "targetName": "ltibootcamp-win"
      },
      "iframe": {
        "height": 890
      }
}

now = int(time())


deep_linking_response = {
    "iss": tool_info['client_id'],
    "aud": content_item_message['iss'],
    "exp": now + 60,
    "iat": now,
    fc("message_type"): "DeepLinkingResponse",
    fc("version"): "1.3.0",
    fc("content_items", s='dl'): [
        duedate_assignment_link
    ]
    
}

# and return to the platform with the selection

deep_linking_response_token = jwt.encode(deep_linking_response, notebook_private_key, 'RS256').decode()

autosubmit_js = """
                var ltiForm = $('<form>');                
                ltiForm.attr('action', '{url}');
                ltiForm.attr('method', 'POST');
                ltiForm.attr('target', 'deeplinking_frame_2');
                $('<input>').attr({{
                    type: 'hidden',
                    name: 'jws_token',
                    value: '{token}'
                }}).appendTo(ltiForm);
                $('#deeplinking_frame_2').before(ltiForm);
                ltiForm.submit();
                ltiForm.remove();
                """

autosubmit_js = autosubmit_js.format(url=deep_linking_return_url, token=deep_linking_response_token)

display(HTML('<iframe id="deeplinking_frame_2" name="deeplinking_frame_2" style="height: 300px; width:100%"></iframe>'))
display(Javascript(data=autosubmit_js, 
                   lib="https://code.jquery.com/jquery-3.3.1.min.js"))

<IPython.core.display.Javascript object>

## Task 2: launch the activity and verify substitution parameters

Now that we have created a link, let's get a student launch and see if on launch we can get the current due date time. Those could indeed have been changed on the platform, or that student may benefit from an actual extension.

In [75]:
# paste the id here of the assignment with dates
resource_link_id='617052aa-433a-11e9-b6a6-024283bd1de6' 

message_hint = '{0}rlid{1}'.format(context_id, resource_link_id)

init_login_request = {
    'iss': platform_url,
    'login_hint': 'student', #value is opaque to tool, ex: tie to end user's platform's session
    'lti_message_hint': message_hint,
    'target_link_uri': 'https://lti.bootcamp/assignment1' #actual URL to launch at the end of the flow, would be the url provided in LtiResourceLink payload if provided
}

orig_state = 'XSRF_session_bound_value_{r}'.format(r=randint(0, 100000))

# Using the same value for the nonce
nonce = orig_state

auth_request_params = {
    'scope': 'open_id', #we only use the flow for OIDC, not to get an access token
    'response_type': 'id_token',
    'client_id': tool_info['client_id'],
    'login_hint': init_login_request['login_hint'],
    'lti_message_hint': init_login_request['lti_message_hint'],
    'response_mode': 'form_post', # with LTI we expect the user to already have a session in the platform
    'nonce': nonce, #a nonce compared to state should fail if replayed
    'state': orig_state, #state is optional
    'redirect_uri': redirect_uri
}

autopost_iframe('resourcelink_with_dates', tool_info['oidc_auth_endpoint'], auth_request_params)

<IPython.core.display.Javascript object>

In [76]:
# Paste id_token here
id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6IjE1NTE5NzUxMDNfMCJ9.eyJub25jZSI6IlhTUkZfc2Vzc2lvbl9ib3VuZF92YWx1ZV83MjUwOCIsImlhdCI6MTU1MjIyNTU0NSwiZXhwIjoxNTUyMjI2MTQ1LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAiLCJhdWQiOiIwIiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vZGVwbG95bWVudF9pZCI6ImRlcGxveW1lbnRfMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL21lc3NhZ2VfdHlwZSI6IkxUSVJlc291cmNlTGlua0xhdW5jaCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3ZlcnNpb24iOiIxLjMuMCIsImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL2xhdW5jaF9wcmVzZW50YXRpb24iOnsiZG9jdW1lbnRfdGFyZ2V0IjoiaWZyYW1lIiwicmV0dXJuX3VybCI6Imh0dHA6Ly9sb2NhbGhvc3Q6NTAwMGh0dHA6Ly9sb2NhbGhvc3Q6NTAwMC8ifSwic3ViIjoiTFRJQkNVXzIxIiwiZ2l2ZW5fbmFtZSI6Ikt5b3UiLCJmYW1pbHlfbmFtZSI6Illldmhlbml5YSIsIm5hbWUiOiJLeW91IFlldmhlbml5YSIsImVtYWlsIjoiS3lvdS5ZZXZoZW5peWFAZXhhbXBsZS5jb20iLCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS9jbGFpbS9yb2xlcyI6WyJodHRwOi8vcHVybC5pbXNnbG9iYWwub3JnL3ZvY2FiL2xpcy92Mi9tZW1iZXJzaGlwI0xlYXJuZXIiXSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vY29udGV4dCI6eyJpZCI6ImJiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNiIsImxhYmVsIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInRpdGxlIjoiTFRJIEJvb3RjYW1wIENvdXJzZSIsInR5cGUiOlsiQ291cnNlU2VjdGlvbiJdfSwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGkvY2xhaW0vcmVzb3VyY2VfbGluayI6eyJpZCI6IjYxNzA1MmFhLTQzM2EtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNiIsInRpdGxlIjoiQXNzaWdubWVudCB3aXRoIERhdGVzIn0sImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL2N1c3RvbSI6eyJsYWIiOiJzaW0zYiIsImR1ZURhdGUiOiIyMDE5LTAzLTE3VDEzOjQyOjQzLjEwNjUzMyIsImNvbnRleHRIaXN0b3J5IjoiJENvbnRleHQuaWQuaGlzdG9yeSJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS1hZ3MvY2xhaW0vZW5kcG9pbnQiOnsic2NvcGUiOlsiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktYWdzL3Njb3BlL2xpbmVpdGVtIiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktYWdzL3Njb3BlL3Njb3JlIiwiaHR0cHM6Ly9wdXJsLmltc2dsb2JhbC5vcmcvc3BlYy9sdGktYWdzL3Njb3BlL3Jlc3VsdC5yZWFkb25seSJdLCJsaW5laXRlbXMiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAvYmJiOWFiZWMtNDBmMy0xMWU5LWI2YTYtMDI0MjgzYmQxZGU2L2xpbmVpdGVtcyIsImxpbmVpdGVtIjoiaHR0cDovL2xvY2FsaG9zdDo1MDAwL2JiYjlhYmVjLTQwZjMtMTFlOS1iNmE2LTAyNDI4M2JkMWRlNi9saW5laXRlbXMvMy9saW5laXRlbSJ9LCJodHRwczovL3B1cmwuaW1zZ2xvYmFsLm9yZy9zcGVjL2x0aS1ucnBzL2NsYWltL25hbWVzcm9sZXNlcnZpY2UiOnsiY29udGV4dF9tZW1iZXJzaGlwc191cmwiOiJodHRwOi8vbG9jYWxob3N0OjUwMDAvYmJiOWFiZWMtNDBmMy0xMWU5LWI2YTYtMDI0MjgzYmQxZGU2L21lbWJlcnNoaXBzIn0sImh0dHBzOi8vcHVybC5pbXNnbG9iYWwub3JnL3NwZWMvbHRpL2NsYWltL3Rvb2xfcGxhdGZvcm0iOnsibmFtZSI6IkxUSSBCb290Y2FtcCBQbGF0Zm9ybSIsImd1aWQiOiJsdGliY19hdF8xNTUxOTc1MTA1In19.ncogCSO-dxO7CmxwdGO15JgFb9Zsrh_z7Q227dgT4VlrnrTDKH_k1MtFJ7yGkq3gZMRtnt5YWiz3-b6zydAYbmsDiev1yGXGx7ksj6qf-ZAoBx3HZc8h6BxfSKD9ooqiF5OCagPf_kcP2KHEI4Ji6YICmCjXwT63SZCjelv78l-rizRuCmJ4JKhLoRgDp3xbu9E1V5TT4kXyDy045fO3t91nsghDvs0HGm9UCavByENxlk61VmCw8D65_1btPNKxQ7YPAcmyhO0L4XIh7FTt3VAvm0vDb5K-fYmy3GF0USM3pwIZsdOawtWWFo8ri038SSgUdpV02IVMZbCfBNfAfw'

jwt_headers = jwt.get_unverified_header(id_token)

launch = jwt.decode(id_token, 
                            platform_keys[jwt_headers['kid']], 
                            jwt_headers['alg'],
                            audience = tool_info['client_id'])

md('Student `{0}` assignment is due on `{1}`'.format(launch['name'], launch[fc('custom')]['dueDate']))
md('The history of this course context is `{0}`'.format(launch[fc('custom')]['contextHistory']))


Student `Kyou Yevheniya` assignment is due on `2019-03-17T13:42:43.106533`

The history of this course context is `$Context.id.history`